In [41]:
import itertools
import numpy as np

def matrix(a, b, match_score=3, gap_cost=2):
    H = np.zeros((len(a) + 1, len(b) + 1), np.int)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else - match_score)
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H

In [42]:
def traceback(H, b, b_='', old_i=0):
    # flip H to get index of **last** occurrence of H.max() with np.argmax()
    H_flip = np.flip(np.flip(H, 0), 1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1))  # (i, j) are **last** indexes of H.max()
    #print('H',H)
    #print('ij',i, j, old_i)
    if H[i, j] == 0:
        return b_, j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_
    #print('b_',b_)
    return traceback(H[0:i, 0:j], b, b_, i)

In [43]:
def smith_waterman(a, b, match_score=3, gap_cost=2):
    a, b = a.upper(), b.upper()
    H = matrix(a, b, match_score, gap_cost)
    b_, pos = traceback(H, b)
    return pos, pos + len(b_)

In [44]:
# prints correct scoring matrix from Wikipedia example
print(matrix('GGTTGACTA', 'TGTTACGG'))

a, b = 'ggttgacta', 'tgttacgg'
H = matrix(a, b)
print(traceback(H, b)) # ('gtt-ac', 1)

a, b = 'GGTTGACTA', 'TGTTACGG'
start, end = smith_waterman(a, b)
print(a[start:end])     # GTTGAC

[[ 0  0  0  0  0  0  0  0  0]
 [ 0  0  3  1  0  0  0  3  3]
 [ 0  0  3  1  0  0  0  3  6]
 [ 0  3  1  6  4  2  0  1  4]
 [ 0  3  1  4  9  7  5  3  2]
 [ 0  1  6  4  7  6  4  8  6]
 [ 0  0  4  3  5 10  8  6  5]
 [ 0  0  2  1  3  8 13 11  9]
 [ 0  3  1  5  4  6 11 10  8]
 [ 0  1  0  3  2  7  9  8  7]]
('gtt-ac', 1)
GTTGAC


/tmp/ipykernel_36185/2084632750.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  H = np.zeros((len(a) + 1, len(b) + 1), np.int)
